# Introduction
In any machine learning task, cleaning or preprocessing the data is as important as model building if not more. And when it comes to unstructured data like text, this process is even more important.
Objective of this kernel is to understand the various text preprocessing steps with code examples.
Some of the common text preprocessing / cleaning steps are:
- Lower Casing
- Spelling Correction
- Remove: 
    - Punctuations
    - Special Chracters
    - Expand Contractions (optinal deoend on the NLP model)
    - Stop words
    - Frequent Words
    - Rare Words
    - URLs
    - HTML tags    
    - Emojis (or convert them to words)
    - Emoticons (or convert them to words)
- Conversion: 
    - Numbers to Words
    - Emojis to Words
    - Emoticons to Words
- Stemming
- Lemmatization 


In [73]:
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
import random
import textwrap

# Download NLTK resources if not already downloaded
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer


wrapper = textwrap.TextWrapper(width=60)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns', None)
random.seed(55)
%matplotlib inline

[nltk_data] Downloading package punkt to /Users/ali/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/ali/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/ali/nltk_data...


# Dataset
> The [Customer Support on Twitter](https://www.kaggle.com/datasets/thoughtvector/customer-support-on-twitter) dataset is a large, modern corpus of tweets and replies to aid innovation in natural language understanding and conversational models, and for study of modern customer support practices and impact.

In [65]:
# Reading just the 'text' column and assuring its data type 
DATA_ADDRESS = "../data/archive/twcs/twcs.csv"
df = pd.DataFrame(
    data=pd.read_csv(DATA_ADDRESS, nrows=1000)['text'].astype(str),
    columns=['text']
    )

# Keep rows with lenght size bigger than threshold
threshold = 200
df['length'] = df.text.apply(len)
df = df[df.length > threshold]

In [68]:
idx = random.randint(0, len(df))
print(wrapper.fill(df.iloc[idx]['text']))

@marksandspencer aren’t require charge 4 a bag: paper bags
shops in airports, or on board trains, aeroplanes or ships
bags which only contain certain items, such as unwrapped
food, raw meat and fish where there is a food safety risk,
prescription medicines, uncovered blades, seeds, bulbs &amp;
🌷s


### Lower Casing

For text featurization (e.g. tf-idf) and avoid duplications to get correct counts and/or save tokens. Not recommended for Part of Speech tagging or Sentiment Analysis.

In [70]:
df.text = df.text.str.lower()
print(wrapper.fill(df.iloc[idx]['text']))

@marksandspencer aren’t require charge 4 a bag: paper bags
shops in airports, or on board trains, aeroplanes or ships
bags which only contain certain items, such as unwrapped
food, raw meat and fish where there is a food safety risk,
prescription medicines, uncovered blades, seeds, bulbs &amp;
🌷s


### Remove Punctuations

Remove punctuations if they don't carry important information for your use case. 
For instance:

```python
> print(string.punctuation)
> !"#$%&\'()*+,-./:;<=>?@[\\]^_{|}~
```

In [113]:
PUNC_TO_REMOVE = string.punctuation
PUNCT_TO_REMOVE_TABLE = str.maketrans('', '', PUNC_TO_REMOVE)

def remove_punctuations(text: str) -> str:
    return text.translate(PUNCT_TO_REMOVE_TABLE)

df.text = df.text.apply(lambda x: remove_punctuations(x))

In [114]:
print(wrapper.fill(df.iloc[idx]['text']))

marksandspencer aren’t require charge 4 bag paper bags shops
airports board trains aeroplanes ships bags contain certain
items unwrapped food raw meat fish food safety risk
prescription medicines uncovered blades seeds bulbs amp 🌷s


### Stop Words

Stopwords are commonly occuring words in a language like 'the', 'a' and so on. They can be removed from the text most of the times, as they don't provide valuable information for downstream analysis. In cases like Part of Speech tagging, we should not remove them as provide very valuable information about the POS.

In [76]:
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text: str) -> str:
    return " ".join([word for word in text.split() if word not in STOPWORDS])

df.text = df.text.apply(lambda x: remove_stopwords(x))

In [77]:
print(wrapper.fill(df.iloc[idx]['text']))

marksandspencer aren’t require charge 4 bag paper bags shops
airports board trains aeroplanes ships bags contain certain
items unwrapped food raw meat fish food safety risk
prescription medicines uncovered blades seeds bulbs amp 🌷s


### Frequent Words

In the previos preprocessing step, we removed the stopwords based on language information. But say, if we have a domain specific corpus, we might also have some frequent words which are of not so much importance to us.

In [116]:
from typing import List
from collections import defaultdict

def find_top_k_frequent_words(text: str, top_k: int, frequent: bool) -> List[str]:
    l_text = text.split()
    hashCount = defaultdict()
    container = [[] for i in range(len(l_text) + 1)]

    for word in l_text:
        hashCount[word] = 1 + hashCount.get(word, 0)
    
    for w, c in hashCount.items():
        container[c].append(w)

    result = []
    start, stop, step = -1, 0, -1
    if not frequent:
        start, stop, step = 1, len(l_text) + 1, 1
    for ws in container[start:stop:step]:
        for w in ws:
            result.append(w)
            if len(result) > top_k - 1:
                return result
                
find_top_k_frequent_words(df.iloc[idx]['text'], 4, True)

['bags', 'food', 'marksandspencer', 'aren’t']

In [107]:
from collections import Counter

cnt = Counter()

for word in df.iloc[idx]['text'].split():
    cnt[word] += 1

[item[0] for item in cnt.most_common(4)]

['bags', 'food', 'marksandspencer', 'aren’t']

### Rare Words

This is very similar to previous preprocessing step but we will remove the rare words from the corpus.

In [119]:
cnt.most_common(4)

[('bags', 2), ('food', 2), ('marksandspencer', 1), ('aren’t', 1)]

In [117]:
find_top_k_frequent_words(df.iloc[idx]['text'], 4, False)

['marksandspencer', 'aren’t', 'require', 'charge']

In [120]:
RARE_WORDS = [item[0] for item in cnt.most_common()[:-4-1:-1]]

In [ ]:
def remove_rare_words(text: str) -> str:
    return " ".join([word for word in text.split() if word not in RARE_WORDS])

### Stemming

In Natural Language Processing (NLP), stemming is the process of reducing a word to its base or root form. The goal of stemming is to reduce inflected or derived words to their word stem, base, or root form, which may not necessarily be a valid word itself. This helps in standardizing words and improving the performance of text analysis tasks such as search engines and text classification.

For example, the words "running," "runner," and "ran" might all be reduced to the root form "run." Similarly, "studies" and "studying" might be reduced to "studi."

Common stemming algorithms include:

- Porter Stemmer: One of the most popular stemming algorithms, known for its simplicity and effectiveness.
- Snowball Stemmer: An improved version of the Porter Stemmer, also known as the Porter2 stemmer.
- Lancaster Stemmer: A more aggressive stemming algorithm compared to Porter and Snowball.

Stemming can help in:

- Search Engines: By stemming words, search engines can match different forms of a word to provide more comprehensive search results.
- Text Classification: Reducing words to their root form can help in grouping similar terms together, making it easier to classify text documents.
- Information Retrieval: Improves the recall rate by matching terms with similar roots.

In [139]:
from nltk.stem.porter import PorterStemmer
from porter2stemmer import Porter2Stemmer
from nltk.stem.lancaster import LancasterStemmer

poster_stemmer = PorterStemmer()
poster2_stemmer = Porter2Stemmer()
lancaster_stemmer = LancasterStemmer()

def stem_words(text: str) -> str:
    l_text = text.split()
    poster_stemmed = " ".join([poster_stemmer.stem(word) for word in l_text])
    poster2_stemmed = " ".join([poster2_stemmer.stem(word) for word in l_text])
    lancaster_stemmed = " ".join([lancaster_stemmer.stem(word) for word in l_text])
    return poster_stemmed, poster2_stemmed, lancaster_stemmed
            
df[["Poster", "Poster2", "Lancaster"]] = df.text.apply(lambda text: pd.Series(stem_words(text)))

Also this porter stemmer is for English language. If we are working with other languages, we can use snowball stemmer. The supported languages for snowball stemmer are

In [124]:
from nltk.stem.snowball import SnowballStemmer
SnowballStemmer.languages

('arabic',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'hungarian',
 'italian',
 'norwegian',
 'porter',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish')

However, stemming has its limitations. It can sometimes lead to overstemming, where different words with different meanings are reduced to the same root, or understemming, where words that should be reduced to the same root are not. In such cases, more sophisticated techniques like lemmatization might be preferred, as lemmatization considers the context and grammar to reduce words to their meaningful base form.

### Lemmatization

Lemmatization is a process in Natural Language Processing (NLP) that reduces words to their base or root form, known as a lemma. Unlike stemming, which simply truncates words to remove prefixes or suffixes, lemmatization considers the context and morphological analysis of the words to convert them into meaningful base forms.

Key Points of Lemmatization
- Context-Aware: Lemmatization uses vocabulary and morphological analysis to correctly transform words. For example, the word "better" is lemmatized to "good," and "am," "are," "is" are all lemmatized to "be."

- Part of Speech (POS) Tagging: Lemmatization often involves part-of-speech tagging, which helps in identifying the correct form of the word. For instance, the word "running" can be lemmatized to "run" (verb) or "running" (noun) based on its usage.

- Meaning Preservation: It ensures that the transformed word (lemma) is a valid word that exists in the language, preserving its meaning and grammatical correctness.

Lemmatization can be implemented using various NLP libraries such as NLTK or spaCy.

#### Lemmatize by NLTK

In [143]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

# Download necessary datasets
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

lemmatizer = WordNetLemmatizer()
wordnet_map = {"N":wordnet.NOUN, "V":wordnet.VERB, "J":wordnet.ADJ, 
               "R":wordnet.ADV}

def lemmatize_word_by_nltk(text: str) -> str:
    pos_tagged_text = nltk.pos_tag(text.split())
    return " ".join([lemmatizer.lemmatize(
        word, wordnet_map.get(pos, wordnet.NOUN)
        ) for word, pos in pos_tagged_text]
        )

df["lemmatized_nltk"] = df.text.apply(lambda text: lemmatize_word_by_nltk(text))


[nltk_data] Downloading package wordnet to /Users/ali/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ali/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


#### Lemmatize by SpaCy

You need to install the spacy model, `en_core_web_sm`, before loading it 
```bash
python -m spacy download en
```

In [153]:
lemmatize_model = spacy.load('en_core_web_sm')

def lemmatize_word_by_spacy(text: str) -> str:
    return " ".join([token.lemma_ for token in lemmatize_model(text)])

df["lemmatized_spacy"] = df.text.apply(lambda text: lemmatize_word_by_spacy(text))

### Processign Emoji's and Emoticons

Processing emojis and emoticons in NLP is important because these elements carry significant emotional and contextual information. You might want to normalize emojis and emoticons to their textual representations to make them easier to handle. This involves converting them into descriptive text. Two main libraries that are handy are: [`emoji`](https://pypi.org/project/emoji/) and [`emot`](https://github.com/NeelShah18/emot?tab=readme-ov-file).

In [155]:
import emoji

def convert_emojis_to_text(text):
    return emoji.demojize(text)

# Example usage
text = "I love pizza 🍕! It's amazing 😍."
normalized_text = convert_emojis_to_text(text)
print(normalized_text)
# Output: "I love pizza :pizza:! It's amazing :heart_eyes:."


I love pizza :pizza:! It's amazing :smiling_face_with_heart-eyes:.


In [160]:
from emot.emo_unicode import EMOTICONS_EMO

def convert_emoticons_to_text(text):
    for e in EMOTICONS_EMO:
        text = text.replace(e, "_".join(EMOTICONS_EMO[e].replace(",","").replace(":","").split()))
    return text

# Example usage
text = "I am happy :-) but sometimes sad :("
normalized_text = convert_emoticons_to_text(text)
print(normalized_text)
# Output: "I am happy happy_face but sometimes sad sad_face"

I am happy Happy_face_smiley but sometimes sad Frown_sad_andry_or_pouting


#### Remove Emoji's and Emoticons

For some applications, it not harmful to remove them if there is no lost of data. However, the best way is converting them to textual data and then apply deduplication methods to avoid redundancies. For instance, removing similar adjacent words/phrases. 

In [ ]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

remove_emoji("game is on 🔥🔥")


def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS_EMO) + u')')
    return emoticon_pattern.sub(r'', text)

remove_emoticons("Hello :-)")

### Process URLs

For most NLP tasks, removing URLs is a good practice to reduce noise and improve model performance. However, always consider the specific requirements of your application and the nature of your data when deciding whether to remove URLs.

#### When to Remove URLs
Here are some reasons and cases why you might want to remove URLs from your text data. 

- Irrelevant Information: URLs often don't contribute meaningful information for text analysis or understanding. They can introduce noise into the dataset.

- Consistency: Removing URLs can help standardize the text data, making it easier to process and analyze. URLs can vary greatly in format and content, which can disrupt the consistency of the dataset.

- Tokenization: URLs can complicate tokenization, especially if they contain special characters or are very long. Removing them simplifies the tokenization process.

- Model Performance: For models focused on semantic analysis, sentiment analysis, topic modeling, or other tasks, URLs usually don't add value and can even degrade model performance by introducing irrelevant features.

- Privacy and Security: URLs can contain sensitive information. Removing them can help in protecting privacy and ensuring that no sensitive data is inadvertently included in the analysis.

You can use regular expressions (regex) to remove URLs from text data. Here's an example in Python using the re library:

In [161]:
def remove_urls(text: str) -> str:
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

#### When Not to Remove URLs

There are scenarios where URLs might contain valuable information:

- URL Analysis: If your task involves analyzing the content or structure of URLs themselves, such as in link analysis, web scraping, or cybersecurity tasks.

- Information Retrieval: If URLs are being used as identifiers or keys to fetch additional data from the web.

- Named Entity Recognition (NER): If you're building models that need to recognize and classify URLs as specific entities within the text.

In such cases, rather than removing URLs, you might choose to preprocess them differently, such as by replacing them with a placeholder token.

## Processing HTML Tags

For most NLP tasks, removing HTML tags is a beneficial preprocessing step that helps in reducing noise, improving text consistency, and enhancing model performance. However, consider the specific needs of your application and data when deciding whether to remove HTML tags. Here are some methods to remove HTML tags:

In [163]:
import re

def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

# Example usage
html_text = "<p>This is a <b>sample</b> text with <a href='https://example.com'>HTML</a> tags.</p>"
cleaned_text = remove_html_tags(html_text)
print(cleaned_text)
# Output: "This is a sample text with HTML tags."


This is a sample text with HTML tags.


In [162]:
from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Example usage
html_text = "<p>This is a <b>sample</b> text with <a href='https://example.com'>HTML</a> tags.</p>"
cleaned_text = remove_html_tags(html_text)
print(cleaned_text)
# Output: "This is a sample text with HTML tags."


This is a sample text with HTML tags.


#### When Not to Remove HTML Tags
There are some scenarios where you might want to preserve HTML tags:

- HTML Analysis: If your task involves analyzing the structure of HTML documents, such as web scraping or studying the use of specific HTML tags.

- Content Extraction: If you need to extract specific elements from the HTML, such as links, images, or metadata.

- Rich Text Processing: If the formatting provided by HTML tags is necessary for understanding the text, such as in rich text processing tasks.


## Process Chat Words Conversion

This is an important text preprocessing step if we are dealing with chat data. People do use a lot of abbreviated words in chat and so it might be helpful to expand those words for our analysis purposes.

Got a good list of chat slang words from this repo. We can use this for our conversion [here](https://github.com/rishabhverma17/sms_slang_translator/tree/master). We can add more words to this list.

## Spelling Correction